In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import numpy as np
import bz2
import _pickle as cPickle

path = 'folder_with_raw_data';

files = ['Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv',
         'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv',
         'Friday-WorkingHours-Morning.pcap_ISCX.csv',
         'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv',
         'Tuesday-WorkingHours.pcap_ISCX.csv',
         'Wednesday-workingHours.pcap_ISCX.csv']

# 0 - whole dataset, 1 - dataset for subsets, 2 - dataset for subsets with others
create_subset  = 2

n = len(files)
finalDF = pd.read_csv(path + files[0])

for i in range(1,n):
    df = pd.read_csv(path + files[i])
    finalDF = pd.concat([finalDF, df])

print("Total rows: " , finalDF.shape[0])

# remove Flow Bytes/s,Flow Packets/s from data frame
finalDF = finalDF.drop(['Flow Bytes/s', ' Flow Packets/s'],axis= 1)

# print("Total columns: " , finalDF.shape[1])

#classes to remove

array = finalDF[' Label'].unique()
n = len(array)
classes_to_remove = []

for i in range(n):
    subset = finalDF[finalDF[' Label'] == array[i]]
    rows = subset.shape[0]
    if rows<=800:
        classes_to_remove.append(array[i])

# print(classes_to_remove)
# remove classes
finalDF = finalDF[~finalDF[' Label'].isin(classes_to_remove)]
print("No. of rows without some classes:", finalDF.shape[0])
print("Total number of features + target: ", finalDF.shape[1])

# encoding for output - Label
le = LabelEncoder()
le.fit(finalDF[' Label'].unique())
finalDF[' Label'] = le.transform(finalDF[' Label'])


if create_subset==1 or create_subset ==2:
    array = finalDF[' Destination Port'].unique()
    n = len(array)
    array.sort()

    print("Number of rows for each destination port: ")

    count_ports = 0
    count_rows = 0
    count_attack_rows = 0
    remove_ports = [];

    for i in range(n):
        current_df = finalDF[finalDF[' Destination Port'] == array[i]]
        rows = current_df.shape[0]
        attack_rows = current_df[current_df[' Label'] != le.transform(['BENIGN'])[0]].shape[0]
        normal_rows = current_df[current_df[' Label'] == le.transform(['BENIGN'])[0]].shape[0]
        if normal_rows>50 and attack_rows>50:
            print(array[i], " -> ", rows, ", attacks: ", attack_rows)
            count_ports = count_ports + 1
            count_rows = count_rows + rows
            count_attack_rows = count_attack_rows + attack_rows
        else:
            remove_ports.append(array[i])

    # print("Total number of used ports: ", count_ports)
    # percentage = '{0:.3g}'.format(count_rows/finalDF.shape[0]*100)
    # print("Total rows covered: ", count_rows, "(", percentage,"%)")
    # percentage_attacks = '{0:.3g}'.format(count_attack_rows/count_rows*100)
    # print("Total attacks: ", count_attack_rows, "(", percentage_attacks,"%)")

    # print("Number of instances: ", finalDF.shape[0] )

data = finalDF


if create_subset == 0:
    # target
    target = np.array(data[' Label'])

    # data
    data = data.drop([' Label'],axis= 1)
    scaler= MinMaxScaler()
    data = scaler.fit_transform(data)

    print("Final features: ", data.shape[1])
    f = bz2.BZ2File("cic_ids_2017.pbz2", "wb")
    cPickle.dump((data,target),f)
    f.close()

if create_subset == 1:
    data = data[~data[' Destination Port'].isin(remove_ports)]

    # target
    target = np.array(data[' Label'])

    # subsets
    subsets = np.array(data[' Destination Port'])

    # data
    data = data.drop([' Label'],axis= 1)
    data = data.drop([' Destination Port'],axis= 1)

    scaler= MinMaxScaler()
    data = scaler.fit_transform(data)

    print("Final features: ", data.shape[1])

    f = bz2.BZ2File("cic_ids_2017_subsets.pbz2", "wb")
    cPickle.dump((data,target,subsets),f)
    f.close()

if create_subset == 2:
    data[' Destination Port'].replace(dict.fromkeys(remove_ports, -1), inplace=True )

    # target
    target = np.array(data[' Label'])

    # subsets
    subsets = np.array(data[' Destination Port'])

    # data
    data = data.drop([' Label'],axis= 1)
    data = data.drop([' Destination Port'],axis= 1)

    scaler= MinMaxScaler()
    data = scaler.fit_transform(data)

    print("Final features: ", data.shape[1])

    f = bz2.BZ2File("cic_ids_2017_subsets_new.pbz2", "wb")
    cPickle.dump((data,target,subsets),f)
    f.close()

Total rows:  2012223
No. of rows without some classes: 2011539
Total number of features + target:  77
Number of rows for each destination port: 
21  ->  11781 , attacks:  8181
22  ->  13498 , attacks:  6140
53  ->  643986 , attacks:  159
80  ->  541594 , attacks:  382566
88  ->  3920 , attacks:  159
135  ->  749 , attacks:  160
139  ->  1921 , attacks:  197
389  ->  4477 , attacks:  159
443  ->  312986 , attacks:  240
445  ->  1195 , attacks:  179
465  ->  2656 , attacks:  160
1124  ->  259 , attacks:  160
3268  ->  1903 , attacks:  160
8080  ->  2610 , attacks:  1421
Final features:  75
